In [1]:
from qiskit_emulator import EmulatorProvider

In [2]:
RUNTIME_PROGRAM = """
import random

from qiskit import transpile
from qiskit.circuit.random import random_circuit

def prepare_circuits(backend):
    circuit = random_circuit(num_qubits=5, depth=4, measure=True,
                            seed=random.randint(0, 1000))
    return transpile(circuit, backend)

def main(backend, user_messenger, **kwargs):
    iterations = kwargs['iterations']
    interim_results = kwargs.pop('interim_results', {})
    final_result = kwargs.pop("final_result", {})
    for it in range(iterations):
        qc = prepare_circuits(backend)
        user_messenger.publish({"iteration": it, "interim_results": interim_results})
        backend.run(qc).result()

    user_messenger.publish(final_result, final=True)
"""

In [3]:
RUNTIME_PROGRAM_METADATA = {
    "max_execution_time": 600,
    "description": "Qiskit test program"
}

PROGRAM_PREFIX = 'qiskit-test'

In [4]:
provider = EmulatorProvider()
provider.runtime.programs()


{}

In [5]:
program_id = provider.runtime.upload_program(RUNTIME_PROGRAM, metadata=RUNTIME_PROGRAM_METADATA)

In [6]:
provider.runtime.pprint_programs()

4f6a88a199dbdf70:
  Name: 4f6a88a199dbdf70
  Description: Qiskit test program
  Version: 0
  Creation date: 25/06/2021 13:10:06
  Max execution time: 600
  Input parameters:
    none
  Interim results:
    none
  Returns:
    none


In [7]:
runtime_program = provider.runtime.program(program_id)
program_inputs = {
    "iterations": 10
}

In [8]:
job = provider.runtime.run(program_id, options=None, inputs=program_inputs)

DEBUG:qiskit_emulator.emulation_executor:creating temp directory at /tmp/tmparv3xcr1
DEBUG:qiskit_emulator.emulation_executor:finished writing to /tmp/tmparv3xcr1/program.py
DEBUG:qiskit_emulator.emulation_executor:finished writing to /tmp/tmparv3xcr1/params.json
DEBUG:qiskit_emulator.emulation_executor:finished writing to /tmp/tmparv3xcr1/executor.py
DEBUG:qiskit_emulator.emulation_executor:starting ['/root/miniconda3/envs/qruntime/bin/python', '/tmp/tmparv3xcr1/executor.py']
DEBUG:qiskit_emulator.emulation_executor:finished executing ['/root/miniconda3/envs/qruntime/bin/python', '/tmp/tmparv3xcr1/executor.py']
DEBUG:qiskit_emulator.emulation_executor:stdout: exit

DEBUG:qiskit_emulator.emulation_executor:stderr: 
DEBUG:qiskit_emulator.emulation_executor:status: sent COMPLETED


In [10]:
job.status()

'Completed'